In [24]:
git_repo = '../../../../'

import sys
import os
import glob

sys.path.insert(0, os.path.join(git_repo, 'BNP_sensitivity/GMM_clustering/'))
sys.path.insert(0, '../../../../LinearResponseVariationalBayes.py/')

# Linear response libraries
import LinearResponseVariationalBayes as vb
import LinearResponseVariationalBayes.SparseObjectives as obj_lib
import LinearResponseVariationalBayes.OptimizationUtils as opt_lib
import LinearResponseVariationalBayes.ModelSensitivity as model_sens

# Local libraries
import gmm_clustering_lib as gmm_utils
import modeling_lib
import functional_sensitivity_lib as fun_sens_lib 
import utils_lib

import matplotlib.pyplot as plt
%matplotlib inline

from copy import deepcopy

from autograd import numpy as np
from autograd import scipy as sp

# import numpy as np
# import scipy as sp

np.random.seed(453453)

import json

# Load original model

In [25]:
fit_dir = './iris_fits_for_nips/'

In [26]:
json_input_file = os.path.join(fit_dir, 'bnp_full_data_fit_alpha3.0_iris.json')

with open(json_input_file, 'r') as fp:
    fit_dict = json.load(fp)
    model = gmm_utils.get_model_from_checkpoint(fit_dict)
    model_ = gmm_utils.get_model_from_checkpoint(fit_dict)
    best_param = model.global_vb_params.get_free()
    kl_hessian = gmm_utils.get_kl_hessian_from_checkpoint(fit_dict)
    prior_param_vec0 = model.prior_params.get_vector()
    
#moment_model = gmm_utils.InterestingMoments(model)


In [27]:
alpha0 = model.prior_params['alpha'].get()
print(alpha0)

[3.]


# Set up linear sensitivity class

In [28]:
# this is analagous to the moment model, but for expected number of clusters
# e_num_clusters_class = gmm_utils.ExpectedNumClustersFromZ(model)
# e_num_clusters_class.set_e_num_clusters_from_free_param(best_param)

# linear_sensitivity = \
#     obj_lib.ParametricSensitivity(\
#         objective_fun =             model.set_z_get_kl,\
#         input_par =                 model.global_vb_params,\
#         output_par =                e_num_clusters_class.e_num_clusters,\
#         hyper_par =                 model.prior_params,\
#         input_to_output_converter = e_num_clusters_class.set_e_num_clusters,\
#         optimal_input_par =         best_param,\
#         objective_hessian =         kl_hessian)


linear_sensitivity = \
    model_sens.ParametricSensitivityLinearApproximation(\
        objective_functor =         model.set_z_get_kl,\
        input_par =                 model.global_vb_params,\
        hyper_par =                 model.prior_params,\
        input_val0 = best_param, 
        hyper_val0 = prior_param_vec0, 
        hess0 = kl_hessian)


# Load refitted models and get linear approximations 

In [29]:
# # Get the expected posterior predictive number of distinct clusters.
# def get_e_num_pred_clusters_from_free_par(free_par, model):
#     model.global_vb_params.set_free(free_par)
#     mu = model.global_vb_params['v_sticks']['mean'].get()
#     sigma = 1 / np.sqrt(model.global_vb_params['v_sticks']['info'].get())
#     n_obs = model.n_obs
#     return modeling_lib.get_e_number_clusters_from_logit_sticks(mu, sigma, n_obs)

# # Get the expected posterior number of distinct clusters.
# def get_e_num_clusters_from_free_par(free_par, model):
#     model.global_vb_params.set_free(free_par)
#     model.set_optimal_z()
#     return modeling_lib.get_e_number_clusters_from_ez(model.e_z)

In [30]:
files = glob.glob(fit_dir +  '/bnp_full_data_fit_alpha*_iris.json')

In [ ]:
e_num_heavy_clusters_class = gmm_utils.ExpectedNumClustersFromZ(model)
e_num_pred_heavy_clusters_class = gmm_utils.ExpectedPredNumClusters(model)

In [ ]:
free_par_refit_list = []
free_par_lr_list = []

e_num_clusters_lr_list = []
e_num_clusters_refit_list = []

e_num_pred_clusters_lr_list = []
e_num_pred_clusters_refit_list = []

alpha_vec = np.zeros(len(files))

for i in range(len(files)):
    print(files[i])
    
    # load perturbed model 
    #json_input_file = os.path.join(fit_dir, files[i])
    json_input_file = os.path.join(files[i])

    with open(json_input_file, 'r') as fp:
        fit_dict = json.load(fp)
        model_refit = gmm_utils.get_model_from_checkpoint(fit_dict)
    
    # prior parameters from refitted model
    alpha_vec[i] = model_refit.prior_params['alpha'].get()
    prior_param_vec_pert = model_refit.prior_params.get_vector() 

    # get free parameters from the refitted model 
    free_par_refit = model_refit.global_vb_params.get_free()
    free_par_refit_list.append(free_par_refit)
    
    # linear approximation for free params
    free_par_lr = linear_sensitivity.predict_input_par_from_hyperparameters(
        prior_param_vec_pert) 
    free_par_lr_list.append(free_par_lr)
    
    # linear approximation for num clusters
    #out_par_lr = linear_sensitivity.predict_output_par_from_hyperparameters(prior_param_vec_pert, linear = False)
#     e_num_pred_clusters_lr_list.append(
#         gmm_utils.get_e_num_pred_clusters_from_free_par(
#             free_par_lr, model_refit))
#     e_num_pred_clusters_refit_list.append(
#         gmm_utils.get_e_num_pred_clusters_from_free_par(
#             free_par_refit, model_refit))

#     e_num_clusters_lr_list.append(
#         gmm_utils.get_e_num_clusters_from_free_par(
#             free_par_lr, model_refit))
#     e_num_clusters_refit_list.append(
#         gmm_utils.get_e_num_clusters_from_free_par(
#             free_par_refit, model_refit))

    e_num_pred_clusters_lr_list.append(
        e_num_pred_heavy_clusters_class.get_e_num_pred_heavy_clusters_from_free_par(
            free_par_lr, threshold = 1 / 50))
    e_num_pred_clusters_refit_list.append(
        e_num_pred_heavy_clusters_class.get_e_num_pred_heavy_clusters_from_free_par(
            free_par_refit, threshold = 1 / 50))

    e_num_clusters_lr_list.append(
        e_num_heavy_clusters_class.get_e_num_heavy_clusters_from_free_par(
            free_par_lr, threshold = 1 / 50)[0])
    e_num_clusters_refit_list.append(
        e_num_heavy_clusters_class.get_e_num_heavy_clusters_from_free_par(
            free_par_refit, threshold = 1 / 50)[0])

    #out_par_lr_list.append(out_par_lr)
    
    # refitted num clusters
    #model_refit.set_optimal_z()
    #out_par_refit = modeling_lib.get_e_number_clusters_from_ez(model_refit.e_z)
    #out_par_refit = modeling_lib.get_e_number_clusters_from_logit_sticks(model_refit)
    #out_par_refit_list.append(out_par_refit)


./iris_fits_for_nips/bnp_full_data_fit_alpha9.5_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha1.5_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha15.0_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha13.0_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha12.0_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha0.5_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha5.0_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha12.5_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha7.0_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha3.0_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha5.5_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha10.0_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha2.0_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha14.5_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha8.5_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha3.5_iris.json
./iris_fits_for_nips/bnp_full_data_fit_alpha7.5_iris.json
./iris_f

### Check the free parameters

In [ ]:
# for i in range(len(free_par_refit_list)): 
#     plt.figure()
#     plt.plot(free_par_lr_list[i]- best_param, free_par_refit_list[i] - best_param, '+')
#     plt.plot(free_par_lr_list[i]- best_param, free_par_lr_list[i]- best_param, '-')
#     plt.title('alpha = {}'.format(alpha_vec[i]))

In [ ]:
model.global_vb_params.set_free(best_param)
model.set_optimal_z()

print(np.sum(model.e_z, axis=0) / model.n_obs)
print(gmm_utils.get_e_num_clusters_from_free_par(best_param, model_refit))

e_z = model.e_z
k = np.shape(e_z)[1]
print(k - np.sum(np.prod(1 - e_z, axis = 0)))

print(k)
print(np.prod(1 - e_z, axis = 0).shape)

# Plot results on expected number of clusters

In [ ]:
plt.plot(alpha_vec, e_num_clusters_lr_list, '+')
plt.plot(alpha_vec, e_num_clusters_refit_list, 'x')

plt.axvline(x=alpha0, linestyle = '--', color = 'red')

plt.xlabel('alpha')
plt.ylabel('expected number of clusters')

plt.legend(('lr', 'refit'))

plt.figure()
plt.plot(alpha_vec, e_num_pred_clusters_lr_list, '+')
plt.plot(alpha_vec, e_num_pred_clusters_refit_list, 'x')

plt.axvline(x=alpha0, linestyle = '--', color = 'red')

plt.xlabel('alpha')
plt.ylabel('expected number of clusters')

plt.legend(('lr', 'refit'))

In [ ]:
print('Done.')

In [ ]:
# np.save('../../../writing/NIPS_2018_BNP_workshop/data_for_figures/alpha_vec', alpha_vec)

In [ ]:
np.save('../../../writing/NIPS_2018_BNP_workshop/data_for_figures/e_num_clusters_thresh3_lr_alpha3', 
        e_num_clusters_lr_list)

In [ ]:
np.save('../../../writing/NIPS_2018_BNP_workshop/data_for_figures/e_num_clusters_thresh3_refit_alpha3', 
        e_num_clusters_refit_list)